# Signal reconstruction from sampled data
This notebook illustrates the process of reconstructing a signal at points other than the sampling points.  The reconstructed signal is band-limited to half of the sampling frequency.  The recnstruction is based on an ideal brick-wall low pass filter.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
from IPython import display
import pylab as pl

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### Plotting function
This function conducts all of the plotting used in the interactive and non-interactive modes.  The samples are always plotted.  If <code>sample_points</code> are specified, then the sinc function for this sample point is shown, and the contribution to the reconstructed value from all of the samples are shown.  If <code>final</code> is true, then the full reconstructed signal is plotted.

In [ ]:
def plot_for(tr, x, x_rec, sample_points, final = False):
    """
        Display the plot for j samples.
        
        INPUT:
            tr            (array-like): timescale for reconstructed waveform
            x             (array-like): random data sequence
            x_rec         (array-like): reconstructed signal
            sample_points (array_like): sample time points
            final               (bool): if it is the final for loop. Default: False.
            
    """

    # Set the figure size
    pl.rcParams["figure.figsize"] = [10.7, 7.8]
    pl.rcParams.update({'font.size': 16})

    # Clear any previous figure
    pl.clf()

    # Plot the input samples
    ax = pl.gca()
    (markerLines, stemLines, baseLines) = ax.stem(np.arange(1, len(x)+1)-1, x,
                                                  label = 'Original samples',
                                                  markerfmt = 'bo',
                                                  bottom = 0,
                                                  use_line_collection = True)
    pl.setp(stemLines, color = 'blue', linewidth=1) 
    pl.setp(baseLines, color = 'black', linewidth=1) 
    markerLines.set_markerfacecolor('none')
    
    # Label the axes
    ax.set_ylim(bottom = 0)
    ax.set_xlim([0, len(x)-1])
    ax.set_xlabel('Time (samples)')
    ax.set_ylabel('Amplitude')
    ax.set_title('Signal reconstruction')
    
    # Loop through each of the sample points to be considered
    for i in range(0,len(sample_points)):
        time_point = sample_points[i]
        
        # Generate the corresponding time scale for the sinc function impulse
        # response of the reconstruction filter
        trs = (np.arange(1, len(sinc_function)+1)/zero_crossing_spacing - 
              2*(number_of_cycles) + (time_point))
        
        # Plot the impulse response shifted to the considered time-point
        ax.plot(trs, sinc_function,
                color = sample_colours[i],
                label = r'$g(%.1f-\tau)$' %time_point)
        
        # Maintain the axis limits to within the sequence only
        pl.xlim([0, len(x)-1])
        pl.ylim(min(min(sinc_function), min(x_rec)),
                max(max(sinc_function), max(x_rec)))

        # Allocate space for the plot of x(nT)g(t-nT), the contribution of each
        # sample to this particular time-point
        stem_data = np.zeros(x.size)
        for stem_point in range(0, len(x)):
            # Get the index corresponding to t-nT
            for j in range(0, len(trs)):
                if trs[j] > (stem_point):
                    stem_index = j
                    break
            # stem_index = find(trs>(stem_point-1),1)
            # and scale the data value, x(nT), by g(t-nT)
            stem_data[stem_point] = x[stem_point]*sinc_function[stem_index]

        # Plot the contribution of each sample to the considered point in time
        (markerLines, stemLines, baseLines) = ax.stem(np.arange(0, len(x)), stem_data, use_line_collection = True)
        pl.setp(stemLines, color = sample_colours[i], linewidth=1) 
        pl.setp(baseLines, color = 'black', linewidth=1) 
        markerLines.set_markerfacecolor('none')
        markerLines.set_markeredgecolor(sample_colours[i])
        
        # Reconstruct the value for the given point in time        
        resampled_value = np.sum(stem_data)
        
        # and display it
        ax.scatter(time_point, resampled_value, 
                   c = sample_colours[i], 
                   s = 60,
                   marker = '*',
                   label = 'Reconstruction at %.1f' % time_point)
        
        
    # Generate the legend, including all of the time points being considered
    pl.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0., prop={'size': 16}) 
    
    ### If flagged in the parameters, on the last time, plot the reconstruction over the whole time period
    if (final == True):
        ax.plot(tr, x_rec,
                label = 'Full Reconstruction',
                color = 'b')
        
        ax.set_xlim([0, len(x)-1])
        pl.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0., prop={'size': 16}) 
        if not is_jupyter(): 
            pl.tight_layout()
            pl.savefig()
             
    pl.show()

### User specified parameters
The following parameters can be specified to sinc function.

Parameter | Meaning
--------- | -------
<code>zero_crossing_spacing</code>|(e.g. 250)
<code>sequence_length</code> | The length of data sequence(e.g. 9)
<code>samples_to_show</code> |Time points for different samples.e.g.  [2, 3.6]）
<code>sample_colours</code> | The colors of different samples(e.g. green, red)
<code>interactive</code> | True for interactive lecture use

In [ ]:
zero_crossing_spacing = 250
sequence_length = 9
samples_to_show = [2, 3.6]
sample_colours = ['green', 'red']
interactive = True

### Compute sinc function
The sinc function is the impulse response that corresponds to an ideal brick-wall filter.

In [ ]:
number_of_cycles = sequence_length - 1

t = (np.arange(0, 4*number_of_cycles*zero_crossing_spacing+1) / (2*zero_crossing_spacing)
     - number_of_cycles)

# ignore floating points error for division by zero.
old_settings=np.seterr(divide='ignore', invalid='ignore')

sinc_function = np.divide(np.sin(2*np.pi*t), (2*np.pi*t))

# And re-enable the warnings
np.seterr(**old_settings);

The value of sinc(0) will be invalid because of the division by 0 - change to 1

In [ ]:
sinc_function[int((len(t)+1)/2 - 1)] = 1

### Generate a data sequence

In [ ]:
x = np.random.rand(sequence_length)

### Compute a high resolution reconstruction
This is constructed from the samples by convolving them with the sinc impulse response and summing the result.

In [ ]:
x_rec = np.zeros(len(x)*zero_crossing_spacing + len(sinc_function))

For each sample in x, add its contribution to the reconstructed waveform

In [ ]:
for i in range(1, len(x)+1):
    (x_rec[(i-1)*zero_crossing_spacing +
    np.arange(1, len(sinc_function)+1) - 1]) = (x_rec[(i-1)*zero_crossing_spacing + 
                                                np.arange(1, len(sinc_function)+1) - 1] + 
                                                x[i-1] * sinc_function)

### Construct a timescale for the reconstruction
This is helpful for plotting the results.

In [ ]:
tr = np.arange(1, len(x_rec)+1)/zero_crossing_spacing - 2*number_of_cycles

### Display samples

In [ ]:
# For interactive sessions, build up the graph element by element
if interactive:
    # The first time around the following loop, no time points are selected,
    # so only the samples are displayed.
    # For each subsequent time around, another time-point is included

    for i in range(-1, len(samples_to_show)):
        ax = plot_for(tr = tr, x = x, x_rec = x_rec, sample_points = samples_to_show[0:i+1])
        
        # Wait for user input
        input("Paused.  Press enter key to continue...")
        display.clear_output(wait=True)

# Finally, plot the reconstructed signal
plot_for(tr = tr, x = x, x_rec = x_rec, sample_points = samples_to_show, final = True)

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.